# 2. collaborative filtering

### Utility Matrix based on products sold and user reviews

A utlity matrix is consists of all possible user-item preferences (ratings) details represented as a matrix. The utility matrix is sparce as none of the users would buy all the items in the list, hence, most of the values are unknown.



In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

import sklearn
from sklearn.decomposition import TruncatedSVD

In [3]:
#Loading the dataset: Amazon product review dataset
amazon_df = pd.read_csv('./ratings_Beauty.csv')
amazon_df = amazon_df.dropna()
print(amazon_df.shape)
amazon_df.head()

(2023070, 4)


UserId   ProductId  Rating   Timestamp
0  A39HTATAQ9V7YF  0205616461     5.0  1369699200
1  A3JM6GV9MNOF9X  0558925278     3.0  1355443200
2  A1Z513UWSAAO0F  0558925278     5.0  1404691200
3  A1WMRR494NWEWV  0733001998     4.0  1382572800
4  A3IAAVS479H7M7  0737104473     1.0  1274227200

In [4]:
# Subset of Amazon Ratings
amazon_df1 = amazon_df.head(10000)

In [6]:
utility_matrix = amazon_df1.pivot_table(values='Rating', index='UserId', columns='ProductId', fill_value=0)
print(utility_matrix.shape)
utility_matrix.head()

(9697, 886)


ProductId              0205616461  0558925278  0733001998  0737104473  \
UserId                                                                  
A00205921JHJK5X9LNP42           0           0           0           0   
A024581134CV80ZBLIZTZ           0           0           0           0   
A03056581JJIOL5FSKJY7           0           0           0           0   
A03099101ZRK4K607JVHH           0           0           0           0   
A0505229A7NSH3FRXRR4            0           0           0           0   

ProductId              0762451459  1304139212  1304139220  130414089X  \
UserId                                                                  
A00205921JHJK5X9LNP42           0           0           0           0   
A024581134CV80ZBLIZTZ           0           0           0           0   
A03056581JJIOL5FSKJY7           0           0           0           0   
A03099101ZRK4K607JVHH           0           0           0           0   
A0505229A7NSH3FRXRR4            0           0           0           0   

ProductId              130414643X  1304146537  ...  B000052YPE  B000052YPF  \
UserId                                         ...                           
A00205921JHJK5X9LNP42           0           0  ...           0           0   
A024581134CV80ZBLIZTZ           0           0  ...           0           0   
A03056581JJIOL5FSKJY7           0           0  ...           0           0   
A03099101ZRK4K607JVHH           0           0  ...           0           0   
A0505229A7NSH3FRXRR4            0           0  ...           0           0   

ProductId              B000052YPG  B000052YPH  B000052YPM  B000052YPU  \
UserId                                                                  
A00205921JHJK5X9LNP42           0           0           0           0   
A024581134CV80ZBLIZTZ           0           0           0           0   
A03056581JJIOL5FSKJY7           0           0           0           0   
A03099101ZRK4K607JVHH           0           0           0           0   
A0505229A7NSH3FRXRR4            0           0           0           0   

ProductId              B000052YPV  B000052YPY  B000052YQ0  B000052YQ2  
UserId                                                                 
A00205921JHJK5X9LNP42           0           0           0           0  
A024581134CV80ZBLIZTZ           0           0           0           0  
A03056581JJIOL5FSKJY7           0           0           0           0  
A03099101ZRK4K607JVHH           0           0           0           0  
A0505229A7NSH3FRXRR4            0           0           0           0  

[5 rows x 886 columns]

In [12]:
#Transposing the matrix
utility_matrix_t = utility_matrix.T
print(utility_matrix_t.shape)
utility_matrix_t.head()

(886, 9697)


UserId      A00205921JHJK5X9LNP42  A024581134CV80ZBLIZTZ  \
ProductId                                                  
0205616461                      0                      0   
0558925278                      0                      0   
0733001998                      0                      0   
0737104473                      0                      0   
0762451459                      0                      0   

UserId      A03056581JJIOL5FSKJY7  A03099101ZRK4K607JVHH  \
ProductId                                                  
0205616461                      0                      0   
0558925278                      0                      0   
0733001998                      0                      0   
0737104473                      0                      0   
0762451459                      0                      0   

UserId      A0505229A7NSH3FRXRR4  A05492663T95KW63BR75K  \
ProductId                                                 
0205616461                     0                      0   
0558925278                     0                      0   
0733001998                     0                      0   
0737104473                     0                      0   
0762451459                     0                      0   

UserId      A059547920Q3LZVFHLPI3  A07410232KYRFR25CIUGJ  \
ProductId                                                  
0205616461                      0                      0   
0558925278                      0                      0   
0733001998                      0                      0   
0737104473                      0                      0   
0762451459                      0                      0   

UserId      A082796624UNM47DSAI6K  A0864963DOAY7LXGS5I6  ...  AZW1HXXYAC15B  \
ProductId                                                ...                  
0205616461                      0                     0  ...              0   
0558925278                      0                     0  ...              0   
0733001998                      0                     0  ...              0   
0737104473                      0                     0  ...              0   
0762451459                      0                     0  ...              0   

UserId      AZWRTJPN7NXT  AZWTXHXZXFAYP  AZYQEFB9Y5N22  AZZHB6U54UDYW  \
ProductId                                                               
0205616461             0              0              0              0   
0558925278             0              0              0              0   
0733001998             0              0              0              0   
0737104473             0              0              0              0   
0762451459             0              0              0              0   

UserId      AZZHJZP4GQPPZ  AZZNK89PXD006  AZZOFVMQC0BJG  AZZQXL8VDCFTV  \
ProductId                                                                
0205616461              0              0              0              0   
0558925278              0              0              0              0   
0733001998              0              0              0              0   
0737104473              0              0              0              0   
0762451459              0              0              0              0   

UserId      AZZTJQ7CQZUD8  
ProductId                  
0205616461              0  
0558925278              0  
0733001998              0  
0737104473              0  
0762451459              0  

[5 rows x 9697 columns]